In [23]:
import numpy as np
import pandas as pd
import MeCab
import json
import re

In [40]:
flatten = lambda l: [i for sub_l in l for i in sub_l]
m_parser = MeCab.Tagger("-u /Users/dontsentouin/mecab-user-dic/compound.dic -Owakati")

def get_IOB_seq(s, annotations):
    '''
    return IOB sequence
    
    Parameters
    ----------
    s : list
        Splitted string
    annotations: list
        list of a splitted annotation string
    '''
    
    # アノテーション箇所が長い方からタグ付けできるようsortする
    annotations.sort(key=len, reverse=True)
    
    tag_seq = ['O'] * len(s)
    
    if len(annotations) is 0:
        return tag_seq
    
    for annotation in annotations:
        if (len(annotation) > len(s)) or (len(annotation) is 0):
            continue
        
        for i in range(0, len(s) - len(annotation) + 1):
            start = i
            end = i + len(annotation)
            if s[start:end] == annotation:
                tag_seq = _tagging(tag_seq, start, end)
                
    return tag_seq

def _tagging(tag_seq, start, end):
    _tag_seq = tag_seq.copy()
    
    # 既にタグが付与されている場合はタグ付けしない．
    if _tag_seq[start:end] != ['O'] * len(_tag_seq[start:end]):
        return _tag_seq
    
    _tag_seq[start] = 'B'
    if len(_tag_seq[start:end]) > 1:
         _tag_seq[start + 1:end] = ['I'] * len(_tag_seq[start + 1:end])

    return _tag_seq

def format_annotations(annotaions):
    '''
    formatting annotation data
        - split sentence
        - parse by MeCab
    
    Parameters
    ----------
    annotations: list
        Annotation data of a attribute
    '''
    
    prep_annotations = [s.strip() for annotaion in annotaions for s in re.findall(r'[^。]+(?:。|$)', annotaion.strip())]
    prep_annotations = [parse(prep_annotation) for prep_annotation in prep_annotations]
    
    return prep_annotations

def parse(s):
    words = []
    node = m_parser.parseToNode(s)
    while node:
        if len(node.surface) is 0:
            node = node.next
            continue

        words.append(node.surface)
        node = node.next
    
    return words

In [24]:
with open("../../data/compound_train.json", 'r') as f:
    raw_train = json.load(f)

In [45]:
train_df = pd.concat([
    pd.read_pickle("../../data/Production_train_repl_compound.pkl")
    , pd.read_pickle("../../data/Production_test_repl_compound.pkl")
]).drop(columns=['POS', 'tag'])
train_df.head()

,_id,label,sentence,title,words,repl_words
0,10166,False,アンモニア (英: ammonia) は分子式が NH 3 で表される無機化合物。,アンモニア,"[アンモニア, (, 英, :, ammonia, ), は, 分子, 式, が, NH, ...","[[title-compound], (, 英, :, [title-compound], ..."
1,10166,False,常温常圧では無色の気体で、特有の強い刺激臭を持つ。,アンモニア,"[常温, 常, 圧, で, は, 無色, の, 気体, で, 、, 特有, の, 強い, 刺...","[常温, 常, 圧, で, は, 無色, の, 気体, で, 、, 特有, の, 強い, 刺..."
2,10166,False,水に良く溶けるため、水溶液（アンモニア水）として使用されることも多く、化学工業では基礎的な窒...,アンモニア,"[水, に, 良く, 溶ける, ため, 、, 水溶液, （, アンモニア水, ）, として,...","[水, に, 良く, 溶ける, ため, 、, 水溶液, （, [compound], ）, ..."
3,10166,False,塩基の程度は水酸化ナトリウムより弱い。,アンモニア,"[塩基, の, 程度, は, 水酸化ナトリウム, より, 弱い, 。]","[塩基, の, 程度, は, [compound], より, 弱い, 。]"
4,10166,False,窒素原子上の孤立電子対のはたらきにより、金属錯体の配位子となり、その場合はアンミンと呼ばれる。,アンモニア,"[窒素, 原子, 上, の, 孤立, 電子, 対, の, はたらき, により, 、, 金属,...","[窒素, 原子, 上, の, 孤立, 電子, 対, の, はたらき, により, 、, 金属,..."


In [27]:
# 属性リスト
attributes = list(raw_train['entry'][0]['Attributes'].keys())
print(attributes)

['ふりがな', '別称', '用途', '種類', '商標名', '特性', '原材料', '製造方法', '生成化合物', 'CAS番号', '化学式', '密度', '融点', '沸点', '示性式']


In [28]:
en_attributes = [
    "furigana"
    , "another_name"
    , "use"
    , "type"
    , "trademark"
    , "property"
    , "raw_material"
    , "production"
    , "formation"
    , "cas"
    , "chemical_formula"
    , "density"
    , "melting"
    , "boiling"
    , "rational_formula"
]

In [49]:
#train_df = pd.concat([train_df, pd.DataFrame(columns=)])
for entry in raw_train['entry']:
    _id = str(entry['WikipediaID'])
    for attr, en_attr in zip(attributes, en_attributes):
        annotations = format_annotations(entry['Attributes'][attr])
        col_name = en_attr + "_tag_seq"
        train_df.loc[train_df._id == _id, col_name] = train_df.loc[train_df._id == _id].words.apply(lambda x: get_IOB_seq(x, annotations))

In [51]:
for attr, en_attr in zip(attributes, en_attributes):
    col_name = en_attr + "_tag_seq"
    print(attr, "Anotations:", len(flatten([entry['Attributes'][attr] for entry in raw_train['entry']])))
    print("B tag:", train_df[col_name].apply(lambda x: x.count('B')).sum())
    print("I tag:", train_df[col_name].apply(lambda x: x.count('I')).sum())
    print("O tag:", train_df[col_name].apply(lambda x: x.count('O')).sum())
    print("")

ふりがな Anotations: 189
B tag: 192
I tag: 457
O tag: 242784

別称 Anotations: 2160
B tag: 1723
I tag: 1185
O tag: 240525

用途 Anotations: 2326
B tag: 2738
I tag: 9233
O tag: 231462

種類 Anotations: 755
B tag: 989
I tag: 1177
O tag: 241267

商標名 Anotations: 215
B tag: 195
I tag: 69
O tag: 243169

特性 Anotations: 1351
B tag: 968
I tag: 1703
O tag: 240762

原材料 Anotations: 1303
B tag: 2291
I tag: 978
O tag: 240164

製造方法 Anotations: 668
B tag: 691
I tag: 9236
O tag: 233506

生成化合物 Anotations: 492
B tag: 695
I tag: 620
O tag: 242118

CAS番号 Anotations: 547
B tag: 37
I tag: 148
O tag: 243248

化学式 Anotations: 616
B tag: 575
I tag: 1219
O tag: 241639

密度 Anotations: 274
B tag: 3
I tag: 14
O tag: 243416

融点 Anotations: 545
B tag: 31
I tag: 70
O tag: 243332

沸点 Anotations: 374
B tag: 21
I tag: 36
O tag: 243376

示性式 Anotations: 49
B tag: 67
I tag: 332
O tag: 243034



In [54]:
train_idx = pd.read_pickle("../../data/Production_train_repl_compound.pkl")._id.astype(str).unique()
test_idx = pd.read_pickle("../../data/Production_test_repl_compound.pkl")._id.astype(str).unique()

In [55]:
train_df.loc[train_df._id.isin(train_idx)].to_pickle("../../data/train_IOB_repl_compound.pkl")
train_df.loc[train_df._id.isin(test_idx)].to_pickle("../../data/test_IOB_repl_compound.pkl")